In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys


import tflearn.data_utils as du
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

DIRECTORY = r"C:\Users\rassa\Documents\Machine Learning A-Z  Udemy\Arabic Handwritten Characters Dataset"



In [11]:
#Load the dataset
#header = 0 because
#this creates a pandas dataframe (basically a table)
train_data = pd.read_csv(DIRECTORY +  '/csvTrainImages 13440x1024.csv', header = None)
train_label = pd.read_csv(DIRECTORY + '/csvTrainLabel 13440x1.csv', header = None)

test_label = pd.read_csv(DIRECTORY +  '/csvTestLabel 3360x1.csv', header = None)


#what happens if we take off header = none ?
#Row number(s) to use as the column names, and the start of the data. Default behavior is to infer the column names: if no 
#names are passed the behavior is identical to header=0 and column names are inferred from the first line of the file, 
#if column names are passed explicitly then the behavior is identical to header=None.
#notice if we do this we have one row less
test_data = pd.read_csv(DIRECTORY +   '/csvTestImages 3360x1024.csv')
print(test_data)

test_data = pd.read_csv(DIRECTORY +   '/csvTestImages 3360x1024.csv', header = None)
print(test_data)
type(train_data)

      0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.919  0.920  \
0     0    0    0    0    0    0    0    0    0    0  ...      0      0   
1     0    0    0    0    0    0    0    0    0    0  ...      0      0   
2     0    0    0    0    0    0    0    0    0    0  ...      0      0   
3     0    0    0    0    0    0    0    0    0    0  ...      0      0   
4     0    0    0    0    0    0    0    0    0    0  ...      0      0   
...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
3354  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3355  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3356  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3357  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3358  0    0    0    0    0    0    0    0    0    0  ...      0      0   

      0.921  0.922  0.923  0.924  0.925  0.926  0.927  0.928  
0         0      0      0      0    

pandas.core.frame.DataFrame

In [12]:
train_data = train_data.iloc[:,:].values.astype('float32')
train_label = train_label.iloc[:,:].values.astype('int32')-1
test_data = test_data.iloc[:,:].values.astype('float32')
test_label = test_label.iloc[:,:].values.astype('int32')-1


In [13]:
train_label = du.to_categorical(train_label,28)

In [14]:
train_data = train_data/255
test_data = test_data/255

In [15]:
train_data = train_data.reshape([-1, 32, 32, 1])
test_data = test_data.reshape([-1, 32, 32, 1])

In [16]:
#train_data, mean1 = du.featurewise_zero_center(train_data)
#test_data, mean2 = du.featurewise_zero_center(test_data)

In [17]:

recognizer = Sequential()
#for any kind of deep learning we use the sequential model in keras then 
#add layers to it
recognizer.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (32,32,1)))
#the first 2 dimensional convolusion layer will have 32 filters. filters or kernels are 
#what we use to extract features. in this case they are of size 5x5 (kernel size)
#in the first layer it is necessary to specify the shape so for our case its a
#32x32 pixel image and since it's black and white then it has only one dimension
#if it was colored then we would have (32,32,3)
#activation='relu' this is rectified linear unit. the output filters or convolced layers
#might contain some negative values so we apply the rectifier function (or other functions)
recognizer.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(MaxPool2D(pool_size=(2,2)))
recognizer.add(Dropout(0.25))


recognizer.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
recognizer.add(Dropout(0.25))


recognizer.add(Flatten())
recognizer.add(Dense(units = 256, input_dim = 1024, activation = 'relu'))
recognizer.add(Dense(units = 256, activation = "relu"))
recognizer.add(Dropout(0.5))
recognizer.add(Dense(28, activation = "softmax"))

In [18]:
recognizer.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 64)       

In [19]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [20]:
recognizer.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [21]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

In [22]:
datagen.fit(train_data)

In [23]:
recognizer.fit_generator(datagen.flow(train_data,train_label, batch_size=100),
                             epochs = 30, verbose = 2, steps_per_epoch=train_data.shape[0] // 100)

C:\Users\rassa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1240: UserWarning: `model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`model.fit_generator` is deprecated and '


Epoch 1/30
134/134 - 32s - loss: 2.4577 - acc: 0.2583
Epoch 2/30
134/134 - 30s - loss: 1.3077 - acc: 0.5576
Epoch 3/30
134/134 - 31s - loss: 0.8862 - acc: 0.6986
Epoch 4/30
134/134 - 32s - loss: 0.6584 - acc: 0.7795
Epoch 5/30
134/134 - 34s - loss: 0.5078 - acc: 0.8329
Epoch 6/30
134/134 - 34s - loss: 0.4265 - acc: 0.8597
Epoch 7/30
134/134 - 33s - loss: 0.3631 - acc: 0.8835
Epoch 8/30
134/134 - 35s - loss: 0.3073 - acc: 0.9044
Epoch 9/30
134/134 - 33s - loss: 0.2730 - acc: 0.9157
Epoch 10/30
134/134 - 33s - loss: 0.2448 - acc: 0.9229
Epoch 11/30
134/134 - 34s - loss: 0.2252 - acc: 0.9262
Epoch 12/30
134/134 - 33s - loss: 0.2085 - acc: 0.9330
Epoch 13/30
134/134 - 33s - loss: 0.1890 - acc: 0.9407
Epoch 14/30
134/134 - 33s - loss: 0.1857 - acc: 0.9410
Epoch 15/30
134/134 - 32s - loss: 0.1759 - acc: 0.9454
Epoch 16/30
134/134 - 32s - loss: 0.1638 - acc: 0.9494
Epoch 17/30
134/134 - 32s - loss: 0.1559 - acc: 0.9534
Epoch 18/30
134/134 - 33s - loss: 0.1576 - acc: 0.9537
Epoch 19/30
134/134

In [24]:
predictions = recognizer.predict(test_data)
predictions = np.argmax(predictions,axis = 1)

C:\Users\rassa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [25]:
cm = confusion_matrix(test_label, predictions)

In [26]:
accuracy = sum(cm[i][i] for i in range(28)) / test_label.shape[0]
print("accuracy = " + str(accuracy))

accuracy = 0.9761904761904762


In [27]:
print(cm)

[[120   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0 119   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0 118   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   2   0   0   0]
 [  0   0   5 112   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   1   0   0   0   0   2   0   0   0]
 [  0   0   0   0 120   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   3 117   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   2 117   0   0   0   0   0   0   0   0   0   0   0
    1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 108   0   9   0   0   0   0   0   0   0   0
    0   0   0   0   1   0   0   0   2   0]
 [  0   0   0   0   0   0   0   3 111   0   3   0   0   0   0   